# Time Series Forecasting with MSTL

In [1]:
# load libraries and data
import os
os.environ["NIXTLA_ID_AS_COL"] = "1" # suppress future warning
import pandas as pd
import matplotlib.pyplot as plt
from statsforecast import StatsForecast
from statsforecast.models import MSTL, AutoARIMA, SeasonalNaive

/Users/tracy.reuter/anaconda3/lib/python3.11/site-packages/statsforecast/core.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
# load and wrangle data (original data source: https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption)
df = pd.read_csv('https://raw.githubusercontent.com/tracyreuter/time-series-forecasting-mstl/main/electric_load_hourly.csv')
df.columns = ['ds', 'y']
df.insert(0, 'unique_id', 'PJM_Load_hourly')
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values(['unique_id', 'ds']).reset_index(drop=True)

In [3]:
# define MSTL model parameters
mstl = MSTL(
    # seasonalities of the time series
    season_length=[24, 24 * 7],
    # model used to forecast trend
    trend_forecaster=AutoARIMA()
)
sf = StatsForecast(
    # models used to forecast time series
    models=[mstl,
    # benchmark against a SeasonalNaive model
    SeasonalNaive(season_length=24)],
    # data are hourly
    freq='h'
)

In [4]:
# split test and train data, using the last 24 hours for testing
df_test = df.tail(24)
df_train = df.drop(df_test.index)

In [5]:
# fit the model to the train data
sf = sf.fit(df=df_train)
# generate forecasts for the test data
# horizon (h) is the length of the test data
forecasts_test = sf.predict(h=len(df_test), level=[90])

In [6]:
# define function to compare model performance visually
def plot_forecasts(y_hist, y_true, y_pred, models):
    _, ax = plt.subplots(1, 1, figsize = (10, 5))
    y_true = y_true.merge(y_pred, how='left', on=['unique_id', 'ds'])
    df_plot = pd.concat([y_hist, y_true]).set_index('ds').tail(24 * 7)
    df_plot[['y'] + models].plot(ax=ax, linewidth=2)
    colors = ['orange', 'green', 'red']
    # omit shading to better see point estimates
    # for model, color in zip(models, colors):
    #     ax.fill_between(df_plot.index,
    #                     df_plot[f'{model}-lo-90'],
    #                     df_plot[f'{model}-hi-90'],
    #                     alpha=.35,
    #                     color=color,
    #                     label=f'{model}-level-90')
    ax.set_title('Actual and Forecasted Load', fontsize=15)
    ax.set_ylabel('Electricity Load', fontsize=15)
    ax.set_xlabel('Timestamp', fontsize=15)
    ax.legend(prop={'size': 10})
    ax.grid()

In [ ]:
# compare model performance visually
plot_forecasts(df_train, df_test, forecasts_test, models=['MSTL', 'SeasonalNaive'])

In [8]:
# import measures for model performance
from datasetsforecast.losses import (
    mae, mape, mase, rmse, smape
)

In [9]:
# define function to compare model performance numerically
def evaluate_performace(y_hist, y_true, y_pred, models):
    y_true = y_true.merge(y_pred, how='left', on=['unique_id', 'ds'])
    evaluation = {}
    for model in models:
        evaluation[model] = {}
        for metric in [mase, mae, mape, rmse, smape]:
            metric_name = metric.__name__
            if metric_name == 'mase':
                evaluation[model][metric_name] = metric(y_true['y'].values,
                                                 y_true[model].values,
                                                 y_hist['y'].values, seasonality=24)
            else:
                evaluation[model][metric_name] = metric(y_true['y'].values, y_true[model].values)
    return pd.DataFrame(evaluation).T

In [ ]:
# compare model performance numerically
evaluate_performace(df_train, df_test, forecasts_test, models=['MSTL', 'SeasonalNaive'])

## Compare MSTL versus Prophet

In [ ]:
# specify prophet model
from prophet import Prophet
# prophet interval_width=0.90 == statsforecast level=[90]
prophet = Prophet(interval_width=0.90)
# fit model
prophet.fit(df_train)
# generate forecasts
future = prophet.make_future_dataframe(periods=len(df_test), freq='h', include_history=False)
forecast_prophet = prophet.predict(future)
# wrangle data
forecast_prophet = forecast_prophet[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_prophet.columns = ['ds', 'Prophet', 'Prophet-lo-90', 'Prophet-hi-90']
forecast_prophet.insert(0, 'unique_id', 'PJM_Load_hourly')
forecast_prophet.head()

In [12]:
# join forecast results
forecasts_test = forecasts_test.merge(forecast_prophet, how='left', on=['unique_id', 'ds'])

In [ ]:
# compare model performance visually
plot_forecasts(df_train, df_test, forecasts_test, models=['MSTL', 'SeasonalNaive', 'Prophet'])

In [ ]:
# compare model performance numerically
evaluate_performace(df_train, df_test, forecasts_test, models=['MSTL', 'Prophet', 'SeasonalNaive'])
# MSTL outperforms SeasonalNaive and Prophet models